In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, to_date, sum

from time import time
import requests

In [2]:
spark.stop()
sc.stop()
conf = SparkConf().setAppName("Jupyter:\n GitHub")
conf = conf.setMaster("spark://david-hdp:7077")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)
spark

In [144]:
df = spark.read.json('git/*')

In [145]:
df.createOrReplaceTempView("git")
df.cache()

DataFrame[_corrupt_record: string, actor: struct<avatar_url:string,display_login:string,gravatar_id:string,id:bigint,login:string,url:string>, created_at: string, id: string, org: struct<avatar_url:string,gravatar_id:string,id:bigint,login:string,url:string>, payload: struct<action:string,before:string,comment:struct<_links:struct<html:struct<href:string>,pull_request:struct<href:string>,self:struct<href:string>>,author_association:string,body:string,commit_id:string,created_at:string,diff_hunk:string,html_url:string,id:bigint,in_reply_to_id:bigint,issue_url:string,line:bigint,original_commit_id:string,original_position:bigint,path:string,position:bigint,pull_request_review_id:bigint,pull_request_url:string,updated_at:string,url:string,user:struct<avatar_url:string,events_url:string,followers_url:string,following_url:string,gists_url:string,gravatar_id:string,html_url:string,id:bigint,login:string,organizations_url:string,received_events_url:string,repos_url:string,site_admin:boolean,s

In [17]:
df.printSchema()

root
 |-- actor: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- display_login: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- org: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _links: struct (nullable = true)
 |    |    |    |-- html: struct (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- pull_request: struct (nul

## Creo una pequeña db sobre la que obtener los lenguajes

In [146]:
lang = spark.sql("SELECT distinct(repo.id), \
                payload.pull_request.base.repo.language as repo_base_lang, \
                payload.pull_request.head.repo.language as repo_head_lang, \
                payload.forkee.language as fork_lang \
            FROM git \
            WHERE \
                payload.pull_request.base.repo.language is not Null OR \
                payload.pull_request.head.repo.language is not Null OR \
                payload.forkee.language is not Null")
lang.persist()
lang.createOrReplaceTempView("lang")

### Obtengo la suma de commits por dia 

In [147]:
result = spark.sql("SELECT to_date(g.created_at) as date, sum(g.payload.size) as size, l.id, \
                        l.repo_base_lang, \
                        l.repo_head_lang, \
                        l.fork_lang \
                FROM git g \
                INNER JOIN lang l on g.repo.id = l.id \
                WHERE g.type = 'PushEvent' \
                GROUP BY g.created_at, l.id, l.repo_base_lang, l.repo_head_lang, l.fork_lang")
result.createOrReplaceTempView("result")

In [143]:
result.show()

+----------+----+--------+--------------+--------------+---------+
|      date|size|      id|repo_base_lang|repo_head_lang|fork_lang|
+----------+----+--------+--------------+--------------+---------+
|2017-01-01|   1|15615485|    JavaScript|    JavaScript|     null|
|2017-01-01|   1|77792086|         Shell|         Shell|     null|
|2017-01-01|   1|75845481|        Python|        Python|     null|
|2017-01-01|   2|76744795|    TypeScript|    TypeScript|     null|
|2017-01-01|   1|19610165|    JavaScript|    JavaScript|     null|
|2017-01-01|   1|68236778|       Arduino|       Arduino|     null|
|2017-01-01|   1|19610165|    JavaScript|    JavaScript|     null|
|2017-01-01|  24|53193142|           CSS|           CSS|     null|
|2017-01-01|   2|72273997|          null|        Python|     null|
|2017-01-01|  14|28008709|          VimL|          VimL|     null|
|2017-01-01|   1|76200114|          HTML|          HTML|     null|
|2017-01-01|  12|76391154|    JavaScript|    JavaScript|     n

In [46]:
take = spark.sql("select * from git where type = 'PushEvent'").first().asDict(recursive=True)

In [149]:
total = df.count()
PushEvent = spark.sql("SELECT repo.id FROM git WHERE type = 'PushEvent'").count()
lang_id = spark.sql("SELECT * FROM lang").count()
commits_w_lang = spark.sql("SELECT g.repo.id FROM git g INNER JOIN lang l ON g.repo.id = l.id WHERE g.type = 'PushEvent'").count()

percent = lambda cnt, total: (cnt*100/total)

row_str = "|{:>13}|{:>15}|{:>15}|{:>15}|{:>15}|{:>15}|"


|             |          Total|    ids /w lang|    Push Events|lang/PushEvents|     lang/total|
|  Raw Numbers|       45192147|         282167|       24776880|           0.20|           0.11|
|  Raw Numbers|         100.00|               |          54.83|          20.44|          11.21|


In [164]:
print(row_str.format("", "Total", "ids /w lang", "Push Events", "lang/PushEvents", "lang/total"))
print("|{:>13}|{:>15}|{:>15}|{:>15}|{:>15.2f}|{:>15.2f}|".format("Raw Numbers", total, lang_id, PushEvent, commits_w_lang, commits_w_lang))
print("|{:>13}|{:>14.2f}%|{:>15}|{:>14.2f}%|{:>14.2f}%|{:>14.2f}%|".format("Percentage", percent(total,total), "", percent(PushEvent,total), percent(commits_w_lang, PushEvent), percent(commits_w_lang, total)))

|             |          Total|    ids /w lang|    Push Events|lang/PushEvents|     lang/total|
|  Raw Numbers|       45192147|         282167|       24776880|     5063785.00|     5063785.00|
|   Percentage|        100.00%|               |         54.83%|         20.44%|         11.21%|


### Guardar DF relacional repo_id y su lenguage

In [150]:
lang.write.json('process_git/lang_id')

### Guardar resultados relacionados

In [ ]:
result.write.mode("append").json('process_git/concurrent_data')